# A step of Algorithm 9.1

In [2]:
using UncNLPTestSet, TRS, LinearAlgebra
nlp = SelectProgram("SROSENBR")
adjdim!(nlp, 100);

┌ Warning: adjdim!: This operation may be unstable
└ @ UncNLPTestSet /Users/daniel/.julia/dev/UncNLPTestSet/src/UncNLPTestSet.jl:127


We initialize and perform our first Hessian sample.

In [3]:
b = 5
n = nlp.n
Sₖ = rand(n, b);
H0 = rand(n,n)
H0 = H0 + H0'
gₖ, hₖ, Yₖ = gHS(nlp, nlp.x0, Sₖ);

The next step is to construct a Quasi-Newton update using:

- $ V = [Y ~ h] = [Y ~ ∇g(x)] = ∇²f(x) U$
- $ U = [S ~ g] $

We first perform a BGFS-Block update.

In [7]:
function bfgs(H::Matrix{<:Real}, U::Matrix{<:Real}, V::Matrix{<:Real}, ϵ::Float64)
    UTVᵀ = U*pinv(U'*V, ϵ)*V'
	E = I - UTVᵀ
	return UTVᵀ + E*H*E
end

bfgs (generic function with 1 method)

In [48]:
Hₖ = bfgs(H0, [Sₖ gₖ], [Yₖ hₖ], 1e-6);

Next, we compute our next direction $p_k$ by solving the trust region subproblem:

$$ p_k = \arg \min_{|p| ≤ Δ_k} \frac{1}{2} p^T H_k^{-1}p + ∇f_k^T p $$

With some work, we obtain an approximate by solving the smaller dimensional problem of:

$$ a_k = \arg \min_{a^TCa ≤ Δ_k^2} $$



In [22]:
Pₖ = [gₖ Hₖ*gₖ Sₖ]
Qₖ = [hₖ gₖ Yₖ]

# collapse information to 7 dimensions
P = Symmetric(Qₖ'*Hₖ*Qₖ) 
h = Qₖ'*gₖ
C = Symmetric(Qₖ'*Hₖ^2*Qₖ)
Δₖ = 1.0

# aBₖ, infoB = trs_boundary_small(P, h, Δₖ, C) this produces the same result - Q: What is the difference?
aₖ, info = trs_small(P, h, Δₖ, C)


([-4.100246163408377e-7; 4.545364454451746e-9; … ; -1.4811317806471308e-9; -3.0453077644503823e-9], TRS.TRSinfo(false, 0, 0, [1618.0872042499466]))

In [44]:
# see: https://github.com/JuliaNLSolvers/Optim.jl/blob/master/src/multivariate/solvers/second_order/newton_trust_region.jl 
η = 0.1 # Nocedal & Wright assert η ∈ [0, 0.25). Optim.jl uses η = 0.1 (link above)
function mₖ(Qₖ, Hₖ, gₖ, a)
	q = Qₖ * a
	return 0.5 * q'*H*q + (H*g)'*q
end

# ndims(Qₖ), ndims(Hₖ), ndims(gₖ), ndims(aₖ)

ma = mₖ(Qₖ, Hₖ, gₖ, vec(aₖ))
m0 = mₖ(Qₖ, Hₖ, gₖ, zeros(length(aₖ)))

0.0

In [46]:
ρ = (obj(nlp, nlp.x0) - obj(nlp, nlp.x0 + vec(Hₖ*Qₖ*aₖ))) / (m0 - ma)

0.5283550592457764

Note, $\rho > \eta$ so we accept our step. Thus, $x_{k+1} = x_k + H_kQ_ka_k$